## Finding parameters and forecasting errors



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

This Notebook is closely based on homework from Physics-151.

#### Problem 1 - Constraining the cosmological parameters using the Planck power spectrum

Planck was the third-generation CMB space telescope, following COBE and WMAP.  Here we will constrain cosmological parameters using the measured angular power spectrum, $C_\ell$, which is related to the correlation function, $C(\theta)$ as:
$$
  C^{TT}(\theta) =  \Big\langle \frac{\delta T}{T}(\hat{n})\frac{\delta T}{T}(\hat{n}') \Big\rangle_{\hat{n} \cdot \hat{n}' = cos\theta}
  = \frac{1}{4\pi}\sum_{\ell=0}^\infty (2\ell+1) C_\ell^{TT} P_\ell(cos\theta)
$$
where $P_\ell$ are Legendre polynomials.

The above expression is specific to the temperature fluctuations, but we can also do a similar analysis for the polarization map of the CMB. (The CMB is polarized because it was scattered off of free electrons during decoupling.) We decompose the polarization pattern in the sky into a curl-free "E-mode" and grad-free "B-mode." 

We will consider the temperature power spectrum $C_\ell^{TT}$, the E-mode power spectrum $C_\ell^{EE} = \frac{1}{2\ell+1}\sum_{m = -\ell}^\ell |\mathrm{a}_{\ell m}^E|^2$, and the temperature-polarization cross-correlation $C_\ell^{TE} = \frac{1}{2\ell+1}\sum_{m = -\ell}^\ell \mathrm{a}_{\ell m}^{T*} \mathrm{a}_{\ell m}^E$. 

The CMB angular power spectrum is usually expressed in terms of $D_\ell = \ell(\ell+1)C_\ell/2\pi$ (in unit of $\mu\,\mathrm{K}^2$).

The predicted angular spectra can be computed with standard "cosmological boltzmann codes" such as [CAMB](https://camb.info/) and [CLASS](http://class-code.net/). We can then fit the theory power spectrum to the data in order to obtain the best-fit parameters or use the variation of the theory to forecast errors on future experiments (and optimize their design).

We will consider the 'standard' 6 parameters of the minimal $\Lambda$CDM model:
$\vec{\theta} = [H_0, \Omega_b h^2, \Omega_c h^2, n_s, A_s, \tau]$. ($H_0$ = Hubble constant, $\Omega_b h^2$ = physical baryon density parameter, $\Omega_c h^2$ = physical cold dark matter density parameter, $n_s$ = scalar spectral index, $A_s$ = curvature fluctuation amplitude, $\tau$ = reionization optical depth.). We provide you with the measured CMB power spectra from Planck Data Release 2 and derivatives of the power spectra about the best-fit model and an initial guess (here chosen to be parameters from the earlier WMAP satellite).

## Best-fit model.

We begin by looking at how to find a best-fit model given the Planck data.  In general this is a multidimensional minimization problem which can be easy or difficult depending upon the ability to compute the Hessian.  In our case it's very close to trivial!  If we start from the "old" WMAP solution then we're not far from the right answer and so we can linearize the dependence of the model upon the parameters.  Then we can do a "single step" $\chi^2$ minimization.

In [ ]:
# First just load the data, models and derivatives.
#
# Measured power spectra from Planck
data = np.loadtxt("EE_measured.dat")
# l (same for all model and measured power spectrum)
ell = data[:,0]
# D_l^EE (measured)
EE_measured = data[:,1]
# and error
error_EE_measured = data[:,2]

# initial estimate of the parameters (theta_{ini}) - from https://lambda.gsfc.nasa.gov/product/map/dr2/params/lcdm_wmap.cfm
H0     = 73.2
ombh2  = 0.02229
omch2  = 0.1054
ns     = 0.958
As10     = 2.347
tau    = 0.089

theta_ini = np.array([H0, ombh2, omch2, ns, As10, tau])

# Model power spectra given \theta_{ini} (calculated at the same ell bins as the measured power spectrum)
data = np.loadtxt("EE_model_at_theta_ini.dat")
# D_l^EE (model)
EE_model = data[:,1]

# Derivative of the power spectra at \theta = \theta_{ini} (calculated at the same ell bins as the measured power spectrum)
data = np.loadtxt("Derivative_EE_at_theta_ini.dat")
# Derivative of D_l^EE with respect to six parameters 
# ([theta1, theta2, theta3, theta4, theta5, theta6] = [H_0, \Omega_b h^2, \Omega_c h^2, n_s, A_s, \tau])
deriv_DlEE_theta1 = data[:,1]
deriv_DlEE_theta2 = data[:,2]
deriv_DlEE_theta3 = data[:,3]
deriv_DlEE_theta4 = data[:,4]
deriv_DlEE_theta5 = data[:,5]/10**9
deriv_DlEE_theta6 = data[:,6]

In [ ]:
# Just as a sanity check, how does the data compare to the model?
# Full ell-range.
plt.figure(figsize=(10,7))
plt.plot(ell, EE_model, 'k-', label = r'$D_\ell\ (\theta_{ini})$')
plt.errorbar(ell, EE_measured, yerr=error_EE_measured, color='steelblue', fmt='.', label = 'Data')
plt.xlim(ell[0], ell[-1])
plt.xlabel(r'$\ell$')
plt.ylabel(r'$D_\ell^{EE}\ [\mu K^2]$')
plt.legend()
plt.show()

# Just the low ells.
plt.figure(figsize=(10,7))
plt.plot(ell[0:29], EE_model[0:29], 'k-', label = r'$D_\ell\ (\theta_{ini})$')
plt.errorbar(ell[0:28], EE_measured[0:28], yerr=error_EE_measured[0:28], color='steelblue', fmt='.', label = 'Data')
plt.xlim(1.9, 30)
plt.xlabel('$\ell$')
plt.ylabel('$D_\ell^{EE}\ [\mu K^2]$')
plt.legend()
plt.show()

In [ ]:
# Now do a single-step minimization to walk to the minimum assuming the
# model depends only linearly on the parameters.
# We need to solve Ax=b with
A = np.vstack([deriv_DlEE_theta1/error_EE_measured, deriv_DlEE_theta2/error_EE_measured, deriv_DlEE_theta3/error_EE_measured, deriv_DlEE_theta4/error_EE_measured, deriv_DlEE_theta5/error_EE_measured, deriv_DlEE_theta6/error_EE_measured]).T
b = (EE_measured-EE_model+deriv_DlEE_theta1*theta_ini[0]+deriv_DlEE_theta2*theta_ini[1]+deriv_DlEE_theta3*theta_ini[2]+deriv_DlEE_theta4*theta_ini[3]+deriv_DlEE_theta5*theta_ini[4]+deriv_DlEE_theta6*theta_ini[5])/error_EE_measured

theta_bestfit = np.linalg.solve(np.dot(A.T,A), np.dot(A.T,b))

print('Best-fit cosmological parameters:')
print('H0 = ', theta_bestfit[0], '\nOmega_b h^2 = ', theta_bestfit[1], '\nOmega_c h^2 = ', theta_bestfit[2], '\nn_s = ', theta_bestfit[3], '\n10^9A_s = ', theta_bestfit[4], '\ntau = ', theta_bestfit[5])

print('')

And this is pretty close to the best-fit solution to the Planck data!

***

#### Problem - Fisher prediction for future CMB surveys

The Fisher information matrix is a standard method in cosmology for designing an experiment, or looking at optimizing combinations of experiments or searching for tensions.  Fisher analyses are fast and simple, allowing one to vary the experiment design and predict the level of the expected error on any given parameter. Below we aim to determine how well a low-noise, high-resolution future CMB survey would do in constraining the cosmological parameters.

The Fisher matrix is defined as the ensemble average of the Hessian of the log-likelihood ($\ln\mathcal{L}$) with respect to the given parameters $\vec{\theta}$:
<br><br>
$$
F_{ij} = -\left\langle\frac{\partial^{2}\ln\mathcal{L}}{\partial\theta_i\partial\theta_j}\right\rangle
$$
We can obtain the covariance matrix, $C$, by inverting the Fisher matrix, $F$:
$$
[C] = [F]^{-1}
$$

We take the model CMB power spectra as our observables (specifically the auto-correlations $D_\ell^{TT}, D_\ell^{EE}$ and cross-correlation $D_\ell^{TE}$ at the best-fit cosmological parameters from Planck, https://arxiv.org/pdf/1502.01589v3.pdf). Then, estimate the Fisher matrix between two parameters $\theta_i$ and $\theta_j$ as:
$$
F_{ij} = \sum_{l} \sum_{k}\frac{1}{(\sigma_l^k)^2}\frac{\partial D^{k}_{\ell}}{\partial\theta_i}\frac{\partial D^{k}_{\ell}}{\partial\theta_j}
$$
where $D_\ell^k = [D_\ell^{TT}, D_\ell^{EE}, D_\ell^{TE}]$, we assume that there is no correlation between them and
$$
(\sigma_\ell^k)^2 = \frac{2}{(2\ell+1) \cdot f_{\rm sky} \cdot \Delta \ell}(D_\ell^k + N_\ell^k)^2
$$
$f_{\rm sky}$ is the fraction of the sky covered by the survey. Assume that $f_{\rm sky} = 1$ for the sake of simplicity. $\Delta\ell$ is the size of $\ell$-bin. Here, we set $\ell_{\rm min} = 2$, $\ell_{\rm max} = 2000$, and we have 92 $\ell$-bins in this range (for $2 \leq \ell < 30$, the power spectra are not binned ($\Delta \ell = 1$), and for $30 \leq \ell < 2000$, they are binned, and the bin size is $\Delta \ell = 30$). We obtain the measured and model power spectrum in the 92 $\ell$-bins.

First take the noise from Planck.


You may wish to look at:
* Fisher Matrix Forecasting Review, https://arxiv.org/pdf/0906.4123.pdf

Load the measurement errors ($\sigma_\ell^{TT}, \sigma_\ell^{EE}, \sigma_\ell^{TE}$), model power spectra ($D_\ell^{TT}, D_\ell^{EE}, D_\ell^{TE}$) and their derivatives with respect to six cosmological parameters. With the measurement errors from Planck, construct the Fisher matrix and the covariance matrix. Evaluate the constraints on six parameters $\sigma(H_0), \sigma(\Omega_bh^2), ... , \sigma(\tau)$ (corresponding to the square root of the diagonal entries of the covariance matrix).

In [ ]:
# Load data

# Best-fit values of the cosmological parameters from https://arxiv.org/pdf/1502.01589v3.pdf
H0     = 67.27
ombh2  = 0.02225
omch2  = 0.1198
ns     = 0.9645
As10     = 2.2065
tau    = 0.079

theta_best_Planck = np.array([H0, ombh2, omch2, ns, As10, tau])


# Planck noise

# sigma_l for D_l^EE
data = np.loadtxt("EE_measured.dat")
# l (same for all power spectrum)
ell = data[:,0]
# and error
error_EE = data[:,2]

# sigma_l for D_l^TT
data = np.loadtxt("TT_measured.dat")
# and error
error_TT = data[:,2]

# sigma_l for D_l^TE
data = np.loadtxt("TE_measured.dat")
# and error
error_TE = data[:,2]


# Model power spectra given theta_best_Planck (calculated at the same ell bins as the measured power spectrum)

# D_l^EE (model)
data = np.loadtxt("EE_model_at_theta_best_Planck.dat")
EE_model_Planck = data[:,1]

# D_l^TT (model)
data = np.loadtxt("TT_model_at_theta_best_Planck.dat")
TT_model_Planck = data[:,1]

# D_l^TE (model)
data = np.loadtxt("TE_model_at_theta_best_Planck.dat")
TE_model_Planck = data[:,1]


# Derivative of the power spectrum given theta_best_Planck (calculated at the same ell bins as the measured power spectrum)

# Derivative of D_l^EE with respect to six parameters 
# ([theta1, theta2, theta3, theta4, theta5, theta6] = [H_0, \Omega_b h^2, \Omega_c h^2, n_s, A_s, \tau])
data = np.loadtxt("Derivative_EE_at_theta_best_Planck.dat")
deriv_DlEE_theta1 = data[:,1]
deriv_DlEE_theta2 = data[:,2]
deriv_DlEE_theta3 = data[:,3]
deriv_DlEE_theta4 = data[:,4]
deriv_DlEE_theta5 = data[:,5]/10**9
deriv_DlEE_theta6 = data[:,6]

# Derivative of D_l^TT with respect to six parameters 
data = np.loadtxt("Derivative_TT_at_theta_best_Planck.dat")
deriv_DlTT_theta1 = data[:,1]
deriv_DlTT_theta2 = data[:,2]
deriv_DlTT_theta3 = data[:,3]
deriv_DlTT_theta4 = data[:,4]
deriv_DlTT_theta5 = data[:,5]/10**9
deriv_DlTT_theta6 = data[:,6]

# Derivative of D_l^TE with respect to six parameters 
data = np.loadtxt("Derivative_TE_at_theta_best_Planck.dat")
deriv_DlTE_theta1 = data[:,1]
deriv_DlTE_theta2 = data[:,2]
deriv_DlTE_theta3 = data[:,3]
deriv_DlTE_theta4 = data[:,4]
deriv_DlTE_theta5 = data[:,5]/10**9
deriv_DlTE_theta6 = data[:,6]

In [ ]:
errs = np.vstack([error_EE, error_TT, error_TE]).T

vec1 = np.vstack([ deriv_DlEE_theta1, deriv_DlTT_theta1, deriv_DlTE_theta1 ]).T
vec2 = np.vstack([ deriv_DlEE_theta2, deriv_DlTT_theta2, deriv_DlTE_theta2 ]).T
vec3 = np.vstack([ deriv_DlEE_theta3, deriv_DlTT_theta3, deriv_DlTE_theta3 ]).T
vec4 = np.vstack([ deriv_DlEE_theta4, deriv_DlTT_theta4, deriv_DlTE_theta4 ]).T
vec5 = np.vstack([ deriv_DlEE_theta5, deriv_DlTT_theta5, deriv_DlTE_theta5 ]).T
vec6 = np.vstack([ deriv_DlEE_theta6, deriv_DlTT_theta6, deriv_DlTE_theta6 ]).T

temp = []
temp.append(vec1); temp.append(vec2); temp.append(vec3); temp.append(vec4); temp.append(vec5); temp.append(vec6)

temp = np.array(temp)

temp2 = temp/errs[np.newaxis, :, :]

fisher = np.dot(temp2.reshape(6,-1), temp2.reshape(6,-1).T)

C = np.linalg.inv(fisher)

In [ ]:
print('1-d constraints:')
print('H0 = %.2f +/- %.2f' % (theta_best_Planck[0], np.sqrt(C.diagonal())[0]), '\nOmega_b h^2 = %.5f +/- %.5f' % (theta_best_Planck[1], np.sqrt(C.diagonal())[1]), '\nOmega_c h^2 = %.4f +/- %.4f' % (theta_best_Planck[2], np.sqrt(C.diagonal())[2]), '\nn_s = %.4f +/- %.4f' % (theta_best_Planck[3], np.sqrt(C.diagonal())[3]), '\n10^9A_s = ', np.around(theta_best_Planck[4], decimals = 12), '+/-', np.around(np.sqrt(C.diagonal())[4], decimals=11), '\ntau = %.5f +/- %.5f' % (theta_best_Planck[5], np.sqrt(C.diagonal())[5]))


Now from the covariance matrix, we can plot 1-d and 2-d constraints on the parameters. (See Fig. 6 in Planck 2015 paper https://arxiv.org/pdf/1502.01589v3.pdf)
<br><br>
<b>1-d constraint</b> (corresponding to the plots along the diagonal in Fig. 6, Planck 2015 paper): <br><br>
First, the $i$th diagonal element of the covariance matrix correspond to $\sigma({\theta_i})^2$. Then, we can plot 1-d constraints on the parameter $\theta_i$ assuming a normal distribution with mean = $(\vec{\theta}_{best-fit})_i$ and variance = $\sigma({\theta_i})^2$.
<br><br>
<b>2-d constraint</b> (off-diagonal plots in Fig. 6, Planck 2015 paper): <br><br>
Consider two parameters $\theta_i$ and $\theta_j$ from $\vec{\theta}$. Now marginalize over other parameters - in order to marginalize over other parameters, you can simply remove those parameters' row and column from the full covariance matrix. (i.e. From the full covariance matrix, you know the variance of all six parameters and their covariances with each other. So build a smaller dimension - 2 x 2 - covariance matrix from this.) - and obtain a $2\times2$ covariance matrix:
<br><br>
$$ \mathrm{C_{ij}} =  \binom{\sigma({\theta_i})^2\ \ \ \ \ \ \mathrm{Cov}({\theta_i, \theta_j})}{\mathrm{Cov}({\theta_i, \theta_j}) \ \ \ \ \ \ \sigma({\theta_j})^2} $$
<br>
Now, we can plot the 2-dimensional confidence region ellipses from this matrix. The lengths of the ellipse axes are the square root of the eigenvalues of the covariance matrix, and we can calculate the counter-clockwise rotation of the ellipse with the rotation angle:
<br><br>
$$ \phi = \frac{1}{2} \mathrm{arctan}\Big( \frac{2\cdot \mathrm{Cov}(\theta_i, \theta_j)}{\sigma({\theta_i})^2-\sigma({\theta_j})^2} \Big) = \mathrm{arctan}(\frac{\vec{v_1}(y)}{\vec{v_1}(x)}) $$
<br>
where $\vec{v_1}$ is the eigenvector with the largest eigenvalue. So we calculate the angle of the largest eigenvector towards the x-axis to obtain the orientation of the ellipse. <br><br> 
Then, we multiply the axis lengths by some factor depending on the confidence level we are interested in. For 68%, this scale factor is $\sqrt{\Delta \chi^2} \approx 1.52$. For 95%, it is $\sqrt{\Delta \chi^2} \approx 2.48$.
<br><br>

In [ ]:
from numpy.linalg import eigvals

from matplotlib.patches import Ellipse
import matplotlib as mpl

from scipy.stats import norm

In [ ]:
# Triangle Plot (Original code by Nicholas Kern)

fig, axes = plt.subplots(6,6,figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

bounds = np.sqrt(C.diagonal()) * 4

p_tex = np.array([r'$H_0$', r'$\Omega_bh^2$',r'$\Omega_ch^2$',r'$n_s$',r'$10^9 A_s$',r'$\tau$'])

for i in range(6):
    for j in range(6):
        ax = axes[i, j]
        if j > i:
            ax.axis('off')
            continue
        elif i == j:
            ax.grid(True)
            xarr = np.linspace(theta_best_Planck[i]-bounds[i],theta_best_Planck[i]+bounds[i],101)
            yarr = norm.pdf(xarr, loc=theta_best_Planck[i], scale=np.sqrt(C[i,i]))
            ax.plot(xarr,yarr)
            ax.set_xlim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_xticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_yticklabels([])
            ax.set_xticklabels([])
        else:
            ax.grid(True)

            CovM = np.array([[C[j,j], C[i,j]],[C[i,j], C[i,i]]])

            eigvec, eigval, u = np.linalg.svd(CovM)

            semimaj = np.sqrt(eigval[0])*2.
            semimin = np.sqrt(eigval[1])*2.

            theta = np.arctan(eigvec[0][1]/eigvec[0][0])

            ell1 = mpl.patches.Ellipse(xy=[theta_best_Planck[j], theta_best_Planck[i]], width=1.52*semimaj, height=1.52*semimin, angle = theta*180/np.pi, facecolor = 'dodgerblue', edgecolor = 'royalblue', label = '68% confidence')
            ell2 = mpl.patches.Ellipse(xy=[theta_best_Planck[j], theta_best_Planck[i]], width=2.48*semimaj, height=2.48*semimin, angle = theta*180/np.pi, facecolor = 'skyblue', edgecolor = 'royalblue', label = '95% confidence')
            #             fig, ax = plt.subplots(figsize=(7,7))

            ax.add_patch(ell2)
            ax.add_patch(ell1)
            
            ax.set_xlim(theta_best_Planck[j]-bounds[j], theta_best_Planck[j]+bounds[j])
            ax.set_ylim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_yticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_xticks([theta_best_Planck[j]-bounds[j]/2, theta_best_Planck[j]+bounds[j]/2])

            
        if j != 0:
            ax.set_yticklabels([])
        if i != 5:
            ax.set_xticklabels([])
        if j == 0 and i !=0:
            ax.set_ylabel(p_tex[i], fontsize=10)
#             ax.set_yticklabels([np.around(theta_best_Planck[i]-bounds[i]/2, decimals=3), np.around(theta_best_Planck[i]+bounds[i]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_yticklabels()]
        if i == 5:
            ax.set_xlabel(p_tex[j], fontsize=10)
#             ax.set_xticklabels([np.around(theta_best_Planck[j]-bounds[j]/2, decimals=3), np.around(theta_best_Planck[j]+bounds[j]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]
    

Now, assume that we have an ideal, zero-noise CMB survey with $N_\ell = 0$. However, we are still instrinsically limited on the number of independent modes we can measure (there are only $(2\ell+1)$ of them) - $C_\ell = \frac{1}{2\ell+1}\sum_{m=-\ell}^{\ell}\langle|a_{\ell m}|^2\rangle$. This leads that we get an instrinsic error (called "cosmic variance") in our estimate of $C_\ell$. So we approximate that
$$
(\sigma_\ell^{EE})^2 = \frac{2}{(2\ell+1) \cdot f_{sky} \cdot \Delta \ell}(D_\ell^{EE})^2,\ \ (\sigma_\ell^{TT})^2 = \frac{2}{(2\ell+1) \cdot f_{sky} \cdot \Delta \ell}(D_\ell^{TT})^2,
$$
$$ (\sigma_l^{TE})^2 = \frac{2}{(2\ell+1) \cdot f_{sky} \cdot \Delta \ell}\frac{(D_\ell^{TE})^2 + D_\ell^{TT}D_\ell^{EE}}{2}
$$

In [ ]:
EE_errs = np.zeros(len(ell))
TT_errs = np.zeros(len(ell))
TE_errs = np.zeros(len(ell))
fsky = 1.
for i in range(len(ell)):
    EE_errs[i] = np.sqrt(2./(2.*ell[i]+1.)/fsky*EE_model_Planck[i]**2)
    TT_errs[i] = np.sqrt(2./(2.*ell[i]+1.)/fsky*TT_model_Planck[i]**2)
    TE_errs[i] = np.sqrt(2./(2.*ell[i]+1.)/fsky*(TE_model_Planck[i]**2 + EE_model_Planck[i]*TT_model_Planck[i])/2.)
    if i > 27:
        EE_errs[i] = np.sqrt(1./30.*2./(2.*ell[i]+1.)/fsky*EE_model_Planck[i]**2)
        TT_errs[i] = np.sqrt(1./30.*2./(2.*ell[i]+1.)/fsky*TT_model_Planck[i]**2)
        TE_errs[i] = np.sqrt(1./30.*2./(2.*ell[i]+1.)/fsky*(TE_model_Planck[i]**2 + EE_model_Planck[i]*TT_model_Planck[i])/2.)


errs = np.vstack([EE_errs, TT_errs, TE_errs]).T

vec1 = np.vstack([ deriv_DlEE_theta1, deriv_DlTT_theta1, deriv_DlTE_theta1 ]).T
vec2 = np.vstack([ deriv_DlEE_theta2, deriv_DlTT_theta2, deriv_DlTE_theta2 ]).T
vec3 = np.vstack([ deriv_DlEE_theta3, deriv_DlTT_theta3, deriv_DlTE_theta3 ]).T
vec4 = np.vstack([ deriv_DlEE_theta4, deriv_DlTT_theta4, deriv_DlTE_theta4 ]).T
vec5 = np.vstack([ deriv_DlEE_theta5, deriv_DlTT_theta5, deriv_DlTE_theta5 ]).T
vec6 = np.vstack([ deriv_DlEE_theta6, deriv_DlTT_theta6, deriv_DlTE_theta6 ]).T

temp = []
temp.append(vec1); temp.append(vec2); temp.append(vec3); temp.append(vec4); temp.append(vec5); temp.append(vec6)

temp = np.array(temp)

temp2 = temp/errs[np.newaxis, :, :]

fisher = np.dot(temp2.reshape(6,-1), temp2.reshape(6,-1).T)

C2 = np.linalg.inv(fisher)

print('1-d constraints:')
print('H0 = %.2f +/- %.5f' % (theta_best_Planck[0], np.sqrt(C2.diagonal())[0]), '\nOmega_b h^2 = %.5f +/- %.7f' % (theta_best_Planck[1], np.sqrt(C2.diagonal())[1]), '\nOmega_c h^2 = %.4f +/- %.7f' % (theta_best_Planck[2], np.sqrt(C2.diagonal())[2]), '\nn_s = %.4f +/- %.5f' % (theta_best_Planck[3], np.sqrt(C2.diagonal())[3]), '\n10^9A_s = ', np.around(theta_best_Planck[4], decimals = 12), '+/-', np.around(np.sqrt(C2.diagonal())[4], decimals=13), '\ntau = %.5f +/- %.6f' % (theta_best_Planck[5], np.sqrt(C2.diagonal())[5]))

# Triangle Plot (Original code by Nicholas Kern)

fig, axes = plt.subplots(6,6,figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

bounds = np.sqrt(C2.diagonal()) * 4

p_tex = np.array([r'$H_0$', r'$\Omega_bh^2$',r'$\Omega_ch^2$',r'$n_s$',r'$10^9A_s$',r'$\tau$'])

for i in range(6):
    for j in range(6):
        ax = axes[i, j]
        if j > i:
            ax.axis('off')
            continue
        elif i == j:
            ax.grid(True)
            xarr = np.linspace(theta_best_Planck[i]-bounds[i],theta_best_Planck[i]+bounds[i],101)
            yarr = norm.pdf(xarr, loc=theta_best_Planck[i], scale=np.sqrt(C2[i,i]))
            ax.plot(xarr,yarr, color = 'crimson')
            ax.set_xlim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_xticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_yticklabels([])
            ax.set_xticklabels([])
        else:
            ax.grid(True)

            CovM = np.array([[C2[i,i], C2[i,j]],[C2[i,j], C2[j,j]]])

            eigvec, eigval, u = np.linalg.svd(CovM)

            semimaj = np.sqrt(eigval[0])*2.
            semimin = np.sqrt(eigval[1])*2.

            theta = np.arctan(eigvec[0][1]/eigvec[0][0])

            ell1 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=1.52*semimaj, height=1.52*semimin, angle = theta*180/np.pi, facecolor = 'orangered', edgecolor = 'crimson', label = '68% confidence')
            ell2 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=2.48*semimaj, height=2.48*semimin, angle = theta*180/np.pi, facecolor = 'salmon', edgecolor = 'crimson', label = '95% confidence')
            #             fig, ax = plt.subplots(figsize=(7,7))

            ax.add_patch(ell2)
            ax.add_patch(ell1)
            
            ax.set_ylim(theta_best_Planck[j]-bounds[j], theta_best_Planck[j]+bounds[j])
            ax.set_xlim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_xticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_yticks([theta_best_Planck[j]-bounds[j]/2, theta_best_Planck[j]+bounds[j]/2])

            
        if j != 0:
            ax.set_yticklabels([])
        if i != 5:
            ax.set_xticklabels([])
        if j == 0 and i != 0:
            ax.set_ylabel(p_tex[i], fontsize=10)
            ax.set_yticklabels([np.around(theta_best_Planck[i]-bounds[i]/2, decimals=3), np.around(theta_best_Planck[i]+bounds[i]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_yticklabels()]
        if j == 0 and i == 1:
            ax.set_xlabel(p_tex[j], fontsize=10)
            ax.set_yticklabels([np.around(theta_best_Planck[i]-bounds[i]/2, decimals=5), np.around(theta_best_Planck[i]+bounds[i]/2, decimals=5) ])
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]
        if i == 5 and j != 1:
            ax.set_xlabel(p_tex[j], fontsize=10)
            ax.set_xticklabels([np.around(theta_best_Planck[j]-bounds[j]/2, decimals=3), np.around(theta_best_Planck[j]+bounds[j]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]
        if i == 5 and j == 1:
            ax.set_xlabel(p_tex[j], fontsize=10)
            ax.set_xticklabels([np.around(theta_best_Planck[j]-bounds[j]/2, decimals=5), np.around(theta_best_Planck[j]+bounds[j]/2, decimals=5) ])
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]
    

Let's compare the Planck results to a "perfect" measurement, with no noise.

In [ ]:
# Triangle Plot (Original code by Nicholas Kern)

fig, axes = plt.subplots(6,6,figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

bounds = np.sqrt(C.diagonal()) * 4

p_tex = np.array([r'$H_0$', r'$\Omega_bh^2$',r'$\Omega_ch^2$',r'$n_s$',r'$10^9A_s$',r'$\tau$'])

for i in range(6):
    for j in range(6):
        ax = axes[i, j]
        if j > i:
            ax.axis('off')
            continue
        elif i == j:
            ax.grid(True)
            xarr = np.linspace(theta_best_Planck[i]-bounds[i],theta_best_Planck[i]+bounds[i],101)
            yarr = norm.pdf(xarr, loc=theta_best_Planck[i], scale=np.sqrt(C[i,i]))
            ax.plot(xarr,yarr)
            yarr2 = norm.pdf(xarr, loc=theta_best_Planck[i], scale=np.sqrt(C2[i,i]))
            yarr2 = yarr2*max(yarr)/max(yarr2)
            ax.plot(xarr,yarr2)
            ax.set_xlim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_xticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_yticklabels([])
            ax.set_xticklabels([])
        else:
            ax.grid(True)

            CovM = np.array([[C[i,i], C[i,j]],[C[i,j], C[j,j]]])

            eigvec, eigval, u = np.linalg.svd(CovM)

            semimaj = np.sqrt(eigval[0])*2.
            semimin = np.sqrt(eigval[1])*2.

            theta = np.arctan(eigvec[0][1]/eigvec[0][0])

            ell1 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=1.52*semimaj, height=1.52*semimin, angle = theta*180/np.pi, facecolor = 'dodgerblue', edgecolor = 'royalblue', label = '68% confidence')
            ell2 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=2.48*semimaj, height=2.48*semimin, angle = theta*180/np.pi, facecolor = 'skyblue', edgecolor = 'royalblue', label = '95% confidence')
            #             fig, ax = plt.subplots(figsize=(7,7))

            ax.add_patch(ell2)
            ax.add_patch(ell1)
            
            CovM2 = np.array([[C2[i,i], C2[i,j]],[C2[i,j], C2[j,j]]])

            eigvec, eigval, u = np.linalg.svd(CovM2)

            semimaj = np.sqrt(eigval[0])*2.
            semimin = np.sqrt(eigval[1])*2.

            theta = np.arctan(eigvec[0][1]/eigvec[0][0])

            ell1 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=1.52*semimaj, height=1.52*semimin, angle = theta*180/np.pi, facecolor = 'orangered', edgecolor = 'crimson', label = '68% confidence')
            ell2 = mpl.patches.Ellipse(xy=[theta_best_Planck[i], theta_best_Planck[j]], width=2.48*semimaj, height=2.48*semimin, angle = theta*180/np.pi, facecolor = 'salmon', edgecolor = 'crimson', label = '95% confidence')
            #             fig, ax = plt.subplots(figsize=(7,7))

            ax.add_patch(ell2)
            ax.add_patch(ell1)
            
            ax.set_ylim(theta_best_Planck[j]-bounds[j], theta_best_Planck[j]+bounds[j])
            ax.set_xlim(theta_best_Planck[i]-bounds[i], theta_best_Planck[i]+bounds[i])
            ax.set_xticks([theta_best_Planck[i]-bounds[i]/2, theta_best_Planck[i]+bounds[i]/2])
            ax.set_yticks([theta_best_Planck[j]-bounds[j]/2, theta_best_Planck[j]+bounds[j]/2])

            
        if j != 0:
            ax.set_yticklabels([])
        if i != 5:
            ax.set_xticklabels([])
        if j == 0 and i !=0:
            ax.set_ylabel(p_tex[i], fontsize=10)
            ax.set_yticklabels([np.around(theta_best_Planck[i]-bounds[i]/2, decimals=3), np.around(theta_best_Planck[i]+bounds[i]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_yticklabels()]
        if i == 5:
            ax.set_xlabel(p_tex[j], fontsize=10)
            ax.set_xticklabels([np.around(theta_best_Planck[j]-bounds[j]/2, decimals=3), np.around(theta_best_Planck[j]+bounds[j]/2, decimals=3) ])
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]
    

***